# 資料挖掘作業 2：吸菸預測模型

本專案透過多個機器學習模型的集成方法來預測吸菸狀態。我們使用了三種強大的梯度提升樹模型：XGBoost、LightGBM 和 CatBoost，並透過 Optuna 進行超參數優化，最後根據驗證集上的 AUC 得分進行加權集成。

## 1. 導入所需套件

首先導入所有需要使用的套件和函式庫。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, PowerTransformer, MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import roc_auc_score
import xgboost as xgb
from xgboost import XGBClassifier
import lightgbm as lgb
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier, Pool
from datetime import datetime
import optuna
import shap
import time
import warnings

warnings.filterwarnings("ignore")
start_time = time.time()

## 2. 資料載入與探索

在這個部分，我們載入訓練和測試資料集，並進行初步的資料探索。

In [ ]:
# 載入資料集
train = pd.read_csv("/kaggle/input/hw2-data/test.csv")
test = pd.read_csv("/kaggle/input/hw2-data/train.csv")
sample_submission = pd.read_csv("/kaggle/input/hw2-data/sample_submission.csv")

# 顯示訓練集的基本資訊
print(f"訓練集形狀: {train.shape}")
print(f"測試集形狀: {test.shape}")
train.head()

## 3. 資料前處理

資料前處理是機器學習流程中非常重要的一環。在這一部分，我們將執行以下步驟：
1. 合併訓練集和測試集以進行一致的特徵工程
2. 處理欄位名稱（替換空格為下劃線）
3. 辨識並分類特徵（類別型和數值型）
4. 進行缺失值處理和特徵轉換

In [ ]:
# 合併訓練集和測試集以統一進行特徵處理
test['smoking'] = np.nan
data = pd.concat([train, test], ignore_index=True)

# 3. Feature Engineering

data['BMI'] = data['weight(kg)'] / (data['height(cm)'] / 100) ** 2
# data['LDL_to_HDL'] = data['LDL'] / (data['HDL'] + 1e-5)
# data['waist_BMI_ratio'] = data['waist(cm)'] / data['BMI']
# data['liver_mean'] = data[['AST', 'ALT', 'Gtp']].mean(axis=1)

# 定義類別型和數值型特徵
categorical_columns = ['hearing(left)', 'hearing(right)', 'Urine protein', 'dental caries']
numerical_columns = [col for col in data.columns if col not in categorical_columns + ['smoking', 'id']
                     and data[col].dtype in ['float64', 'int64']]

# 顯示分類後的特徵數量
print(f"類別型特徵數量: {len(categorical_columns)}")
print(f"數值型特徵數量: {len(numerical_columns)}")

### 3.1 缺失值處理與特徵轉換

我們使用以下方法處理資料：
1. 使用 SimpleImputer 填補缺失值（以中位數填補）
2. 應用 Yeo-Johnson 變換來處理偏態分佈
3. 使用 MinMaxScaler 將數值特徵縮放到相同範圍

In [ ]:

# 應用 Power Transform 處理偏態分佈
power_transformer = PowerTransformer(method='yeo-johnson')
data[numerical_columns] = power_transformer.fit_transform(data[numerical_columns])

# 特徵縮放
scaler = MinMaxScaler()
data[numerical_columns] = scaler.fit_transform(data[numerical_columns])

### 3.2 類別型特徵編碼與特徵工程

在這個部分，我們將：
1. 對類別型特徵進行 One-Hot 編碼
2. 應用 KMeans 聚類作為特徵工程的一部分，創建新的聚類特徵

In [ ]:
categorical_columns

In [ ]:
# One-Hot 編碼
encoder = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')
encoded = encoder.fit_transform(data[categorical_columns])
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(categorical_columns), index=data.index)
data = data.drop(columns=categorical_columns)
data = pd.concat([data, encoded_df], axis=1)

In [ ]:

# 使用 KMeans 進行聚類特徵工程
kmeans = KMeans(n_clusters=5, random_state=42)
data['kmeans_cluster'] = kmeans.fit_predict(data[numerical_columns])

# 檢視特徵工程後的資料
print(f"處理後的特徵數量: {data.shape[1]}")

In [ ]:
print(data.columns)

## 4. 資料分割

將資料分割為訓練集、驗證集和測試集。這是模型訓練和評估的重要步驟。

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# 1. 讀取資料（根據你 Kaggle 加入的資料集名稱）
train = pd.read_csv("/kaggle/input/hw2-data/train.csv")
test = pd.read_csv("/kaggle/input/hw2-data/test.csv")

# 2. 標記訓練集長度，用來之後切分
train_length = len(train)

# 3. 合併資料
data = pd.concat([train, test], ignore_index=True)

# 4. 刪除不需要的特徵
features_to_drop = ['hearing(left)_2.0', 'hearing(right)_2.0', 'eyesight(left)', 'eyesight(right)','BMI','Urine protein_2.0','Urine protein_3.0', 'Urine protein_4.0', 'Urine protein_5.0','kmeans_cluster']
data = data.drop(columns=features_to_drop, errors='ignore')

# 5. 切分訓練與測試資料
X = data.iloc[:train_length].drop(columns=['smoking', 'id'], errors='ignore')
y = data.iloc[:train_length]['smoking'].astype(int)
X_test = data.iloc[train_length:].drop(columns=['smoking', 'id'], errors='ignore')

# 6. 拆分訓練與驗證集
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.1, stratify=y, random_state=42
)

# 7. 顯示形狀確認
print(f"訓練集形狀: {X_train.shape}")
print(f"驗證集形狀: {X_val.shape}")
print(f"測試集形狀: {X_test.shape}")


## 5. 模型超參數優化與訓練

在這個部分，我們將使用 Optuna 來為三種不同的梯度提升樹模型尋找最佳的超參數。超參數優化是提高模型性能的關鍵步驟。

### 5.1 XGBoost 模型優化與訓練

XGBoost 是一種高效能的梯度提升樹實現，特別適合結構化/表格式資料。

In [ ]:
# ✅ 修正後的 XGBoost + Optuna + SHAP 流程
# 1️⃣ 避免資料洩漏：CV 只使用 X_train
# 3️⃣ n_estimators 上限擴至 1000
# 4️⃣ 保留 X_val 作為驗證集
# ✅ 使用 pandas DataFrame

import optuna
import numpy as np
import xgboost as xgb
import shap
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# # 假設 data, train 已定義並預處理
# train_length = len(train)
# X = data.iloc[:train_length].drop(columns=['smoking', 'id'], errors='ignore')
# X_test = data.iloc[train_length:].drop(columns=['smoking', 'id'], errors='ignore')
# y = data.iloc[:train_length]['smoking'].astype(int)

# # ✅ 分割資料，保留 X_val 為未見驗證集
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, stratify=y, random_state=42)

#-----------------------------
# 1. Define Optuna objective function
#-----------------------------
def objective_xgb(trial):
    params = {
        'n_estimators': 500,  # 
        'max_depth': trial.suggest_int('max_depth', 3, 6),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 5, 15),
        'gamma': trial.suggest_float('gamma', 1, 10),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.5, 10.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 1.0, 10.0),
        'tree_method': 'hist',
        'device': 'cuda',
        'eval_metric': 'auc'
    }

    aucs = []
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    for train_idx, val_idx in skf.split(X_train, y_train):  # ✅ 用 X_train 避免資料洩漏
        X_fold_train, X_fold_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_fold_train, y_fold_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

        model = xgb.XGBClassifier(**params)
        model.fit(
            X_fold_train,
            y_fold_train,
            eval_set=[(X_fold_val, y_fold_val)],
            early_stopping_rounds=50,
            verbose=False
        )

        preds = model.predict_proba(X_fold_val)[:, 1]
        aucs.append(roc_auc_score(y_fold_val, preds))

    mean_auc = np.mean(aucs)
    trial.set_user_attr("params", params)
    trial.set_user_attr("mean_auc", mean_auc)
    return mean_auc

#-----------------------------
# 2. Run Optuna optimization
#-----------------------------
study_xgb = optuna.create_study(direction='maximize')
study_xgb.optimize(objective_xgb, n_trials=30)

print("Best XGBoost Parameters:")
print(study_xgb.best_params)
print(f"Best AUC: {study_xgb.best_value:.4f}")

#-----------------------------
# 3. Train Final Model
#-----------------------------

best_xgb = xgb.XGBClassifier(
    **study_xgb.best_params,
    n_estimators=500,
    tree_method='hist',
    device='cuda',
    eval_metric='auc',
    use_label_encoder=False
)

best_xgb.fit(
    X_train,
    y_train,
    eval_set=[(X_train, y_train), (X_val, y_val)],
    early_stopping_rounds=50,
    verbose=False  # ❌ 拿掉 eval_metric 避免衝突
)


#-----------------------------
# 4. Draw learning curve
#-----------------------------
results = best_xgb.evals_result()

plt.figure(figsize=(8, 5))
plt.plot(results['validation_0']['auc'], label='Train AUC')
plt.plot(results['validation_1']['auc'], label='Validation AUC')
plt.xlabel('Boosting Round')
plt.ylabel('AUC Score')
plt.title('XGBoost Learning Curve')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

#-----------------------------
# 5. Predict on validation set & draw confusion matrix
#-----------------------------
xgb_preds = best_xgb.predict_proba(X_val)
xgb_labels = np.argmax(xgb_preds, axis=1)
cm = confusion_matrix(y_val, xgb_labels)

plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('XGBoost Confusion Matrix (Validation Set)')
plt.tight_layout()
plt.show()

#-----------------------------
# 6. Feature Importance Plot
#-----------------------------
plt.figure(figsize=(10, 6))
xgb.plot_importance(best_xgb, max_num_features=15, importance_type='gain', height=0.5)
plt.title("XGBoost Feature Importance (Top 15, by Gain)")
plt.tight_layout()
plt.show()

#-----------------------------
# 7. SHAP Analysis (Global Explanation)
#-----------------------------
explainer = shap.Explainer(best_xgb, X_train, feature_names=X_train.columns)
shap_values = explainer(X_val)
shap.plots.beeswarm(shap_values, max_display=15)

### 5.2 LightGBM 模型優化與訓練

LightGBM 是一個高效、低記憶體佔用的梯度提升框架，使用基於直方圖的分割尋找策略，適合大型資料集。

In [ ]:
# ✅ 修正後的 LightGBM + Optuna + SHAP 流程
# 1️⃣ 避免資料洩漏（只使用 X_train 做 CV）
# 2️⃣ 使用 pandas DataFrame（非 .values）
# 3️⃣ n_estimators 上限提至 1000
# 4️⃣ 保留 X_val 做最終驗證評估

import optuna
import numpy as np
import pandas as pd
from lightgbm import LGBMClassifier
import lightgbm as lgb
import shap
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# 假設 data, train 已定義
# features_to_drop = [...]
# data = data.drop(columns=features_to_drop, errors='ignore')
# train_length = len(train)
# X = data.iloc[:train_length].drop(columns=['smoking', 'id'], errors='ignore')
# X_test = data.iloc[train_length:].drop(columns=['smoking', 'id'], errors='ignore')
# y = data.iloc[:train_length]['smoking'].astype(int)

# # ✅ 分割訓練與驗證集，保留 X_val 評估用
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, stratify=y, random_state=42)

#-----------------------------
# 1. Define Optuna Objective Function
#-----------------------------
def objective_lgb(trial):
    params = {
        'objective': 'binary',
        'metric': 'auc',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'device_type': 'gpu',
        'n_estimators': 500,  # ✅ 提高上限為 1000
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'num_leaves': trial.suggest_int('num_leaves', 20, 64),
        'max_depth': trial.suggest_int('max_depth', 3, 6),
        'min_child_samples': trial.suggest_int('min_child_samples', 20, 100),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.5, 10.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 1.0, 10.0)
    }

    aucs = []
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    for train_idx, val_idx in skf.split(X_train, y_train):  # ✅ 避免洩漏，使用 X_train
        X_fold_train, X_fold_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_fold_train, y_fold_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

        model = LGBMClassifier(**params)
        model.fit(
            X_fold_train, y_fold_train,
            eval_set=[(X_fold_val, y_fold_val)],
            eval_metric='auc',
            callbacks=[lgb.early_stopping(50)]
        )

        preds = model.predict_proba(X_fold_val)[:, 1]
        aucs.append(roc_auc_score(y_fold_val, preds))

    return np.mean(aucs)

#-----------------------------
# 2. Run Optuna Optimization
#-----------------------------
study_lgb = optuna.create_study(direction='maximize')
study_lgb.optimize(objective_lgb, n_trials=30)

print("Best LightGBM Parameters:")
print(study_lgb.best_params)
print(f"Best AUC: {study_lgb.best_value:.4f}")

#-----------------------------
# 3. Train Final Model
#-----------------------------
best_lgb = LGBMClassifier(
    **study_lgb.best_params,
    n_estimators=500,
    verbosity=-1,
    device_type='gpu'
)

# ✅ 最終訓練只用 X_train，評估用 X_val
best_lgb.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_val, y_val)],  # ✅ 加入 train
    eval_metric='auc',
    callbacks=[lgb.early_stopping(50)]
)



#-----------------------------
# 4. Draw Learning Curve
#-----------------------------
results = best_lgb.evals_result_

plt.figure(figsize=(8, 5))
plt.plot(results['training']['auc'], label='Train AUC')         # ✅ eval_set[0]
plt.plot(results['valid_1']['auc'], label='Validation AUC') 
plt.xlabel('Boosting Round')
plt.ylabel('AUC Score')
plt.title('LightGBM Learning Curve')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

#-----------------------------
# 5. Predict & Confusion Matrix
#-----------------------------
lgb_preds = best_lgb.predict_proba(X_val)
lgb_labels = (lgb_preds[:, 1] >= 0.5).astype(int)

cm = confusion_matrix(y_val, lgb_labels)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('LightGBM Confusion Matrix (Validation Set)')
plt.tight_layout()
plt.show()

#-----------------------------
# 6. Feature Importance
#-----------------------------
plt.figure(figsize=(10, 6))
lgb.plot_importance(best_lgb, max_num_features=15, importance_type='gain', height=0.5)
plt.title('LightGBM Feature Importance (Top 15, by Gain)')
plt.tight_layout()
plt.show()

#-----------------------------
# 7. SHAP Analysis
#-----------------------------
explainer = shap.Explainer(best_lgb, X_train, feature_names=X_train.columns)
shap_values = explainer(X_val)
shap.plots.beeswarm(shap_values, max_display=15)



### 5.3 CatBoost 模型優化與訓練

CatBoost 是一種高效能的梯度提升樹實現，尤其擅長處理類別型特徵，並自動處理缺失值。

In [ ]:
import optuna
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import shap
#-----------------------------
# 1. Define Optuna Objective Function (✅ 使用 X_train 進行 CV 調參)
#-----------------------------
def objective_cat(trial):
    params = {
        'iterations': 500, 
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'depth': trial.suggest_int('depth', 3, 6),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 3.0, 10.0),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.2, 1.0),
        'border_count': trial.suggest_int('border_count', 64, 128),
        'eval_metric': 'AUC',
        'random_seed': 42,
        'task_type': 'CPU',
        'od_type': 'Iter',
        'od_wait': 50,
        'verbose': 0
    }

    aucs = []
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    for train_idx, val_idx in skf.split(X_train, y_train):  # ✅ 用 X_train 進行 5-fold CV
        X_fold_train, X_fold_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_fold_train, y_fold_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

        train_pool = Pool(X_fold_train, y_fold_train)
        val_pool = Pool(X_fold_val, y_fold_val)

        model = CatBoostClassifier(**params)
        model.fit(train_pool, eval_set=val_pool, use_best_model=True, verbose=False)

        preds = model.predict_proba(X_fold_val)[:, 1]
        auc = roc_auc_score(y_fold_val, preds)
        aucs.append(auc)

    return np.mean(aucs)

#-----------------------------
# 2. Run Optuna Optimization
#-----------------------------
study_cat = optuna.create_study(direction='maximize')
study_cat.optimize(objective_cat, n_trials=30)

print("Best CatBoost Parameters:")
print(study_cat.best_params)
print(f"Best AUC: {study_cat.best_value:.4f}")

#-----------------------------
# 3. Train Final Model on X_train (✅ 不使用整個 X 以避免資料洩漏)
#-----------------------------
best_cat = CatBoostClassifier(
    **study_cat.best_params,
    iterations=500,
    eval_metric='AUC',
    task_type='CPU',
    random_seed=42,
    use_best_model=True,
    verbose=False
)

best_cat.fit(X_train, y_train, eval_set=(X_val, y_val))


#-----------------------------
# 4. Draw Learning Curve
#-----------------------------
cat_log = best_cat.get_evals_result()

plt.figure(figsize=(8, 5))
plt.plot(cat_log['learn']['Logloss'], label='Train Logloss')
plt.plot(cat_log['validation']['Logloss'], label='Validation Logloss')
plt.xlabel('Iteration')
plt.ylabel('Logloss')
plt.title('CatBoost Learning Curve')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

#-----------------------------
# 5. Predict & Confusion Matrix
#-----------------------------
cat_preds = best_cat.predict_proba(X_val)[:, 1]
cat_labels = (cat_preds >= 0.5).astype(int)

cm = confusion_matrix(y_val, cat_labels)

plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Oranges')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('CatBoost Confusion Matrix (Validation Set)')
plt.tight_layout()
plt.show()

#-----------------------------
# 6. Feature Importance
#-----------------------------
plt.figure(figsize=(10, 6))
feat_importance = best_cat.get_feature_importance(prettified=True)
feat_importance_top = feat_importance.sort_values(by='Importances', ascending=False).head(15)

plt.barh(feat_importance_top['Feature Id'], feat_importance_top['Importances'])
plt.gca().invert_yaxis()
plt.xlabel('Importance')
plt.title('CatBoost Feature Importance (Top 15)')
plt.tight_layout()
plt.show()

#-----------------------------
# 7. SHAP Summary Plot
#-----------------------------
explainer = shap.Explainer(best_cat)
shap_values = explainer(X_val)
shap.plots.beeswarm(shap_values, max_display=15)


## 6. 模型集成與預測

在這部分，我們將根據各個模型在驗證集上的 AUC 表現，進行自動加權集成，結合三個模型的預測結果。

In [ ]:
cat_test_preds = best_cat.predict_proba(X_test)[:, 1]
lgb_test_preds = best_lgb.predict_proba(X_test)[:, 1]
xgb_test_preds = best_xgb.predict_proba(X_test)[:, 1]
final_preds = (cat_test_preds + lgb_test_preds + xgb_test_preds) / 3


In [ ]:
from datetime import datetime

# 1. 用預測結果填入 sample_submission
sample_submission['smoking'] = final_preds

# 2. 存成 timestamp 命名的副本（可備份多個版本）
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
filename = f"submission_{timestamp}.csv"

# 3. 正式輸出給 Kaggle 平台（會自動出現下載按鈕）
sample_submission.to_csv("/kaggle/working/submission.csv", index=False)

# 4. 額外備份一份有時間戳的副本（選用）
sample_submission.to_csv(filename, index=False)